In [1]:
import pandas as pd

df = pd.read_json('./dataset_complete_01302025.jsonl', lines=True)

In [2]:
df.head()

,filename,text,docket_num,court,arguments,token_count,cleaned_text_alt,cleaned_content_alt,cleaned_args,url,arg_structure,sections_final
0,Docket20-5279_Brief007.pdf,No. 20-5279 \n \nIN THE \nSupreme Court of th...,20-5279,SCOTUS,I. THE RULE OF LENITY SHOULD BE APPLIED RIGORO...,9067,No. 20-5279 \n \nIN THE \nSupreme Court of th...,.................................................,[I. THE RULE OF LENITY SHOULD BE APPLIED RIGOR...,https://www.supremecourt.gov/docket/docketfile...,"[{'index': 0, 'header': 'I. THE RULE OF LENITY...",[{'section_id': 'f99d63c9-a024-4844-bc96-d3855...
1,Docket20-5279_Brief008.pdf,No. 20-5279 \nIN THE \nSupreme Court of the Un...,20-5279,SCOTUS,I. AN EXPANSIVE INTERPRETATION OF THE ARMED CA...,7180,No. 20-5279 \nIN THE \nSupreme Court of the Un...,No. 20-5279 \nIN THE \nSupreme Court of the Un...,[I. AN EXPANSIVE INTERPRETATION OF THE ARMED C...,https://www.supremecourt.gov/docket/docketfile...,"[{'index': 0, 'header': 'I. AN EXPANSIVE INTER...",[{'section_id': '55cd7d78-8f13-4a87-9f0e-74af3...
2,Docket20-5279_Brief009.pdf,\n No. 20-5279 \nIn the Supreme Court of the...,20-5279,SCOTUS,Petitioner's 1997 burglary convictions are for...,15564,\n No. 20-5279 \nIn the Supreme Court of the...,.............................. ..................,[Petitioner's 1997 burglary convictions are fo...,https://www.supremecourt.gov/docket/docketfile...,"[{'index': 0, 'header': 'Petitioner's 1997 bur...",[{'section_id': '33c2641c-3032-4b5b-a459-78328...
3,Docket20-5279_Brief010.pdf,\n \n \n \n \n \nNo. 20-5279 \n \n In the Sup...,20-5279,SCOTUS,I. The Text of the 'Occasions' Clause Supports...,7537,\n \n \n \n \n \nNo. 20-5279 \n \n In the Sup...,\n \n \n \n \n \nNo. 20-5279 \n \n In the Sup...,[I. The Text of the 'Occasions' Clause Support...,https://www.supremecourt.gov/docket/docketfile...,"[{'index': 0, 'header': 'I. The Text of the 'O...",[{'section_id': '47a22f17-e94e-4393-996c-d2a6f...
4,Docket20-828_Brief001.pdf,\n No. 20-828 \n=============================...,20-828,SCOTUS,A. The Ninth Circuit's Decision Must Be Revers...,2566,\n No. 20-828 \n=============================...,.................................................,[A. The Ninth Circuit's Decision Must Be Rever...,https://www.supremecourt.gov/docket/docketfile...,"[{'index': 0, 'header': 'A. The Ninth Circuit'...",[{'section_id': '69456dca-d4bf-40da-9a3a-cbf50...


In [3]:
import re

def count_citations(annotated_text, max_distance=5):
    if not isinstance(annotated_text, str):
        return 0, 0  # Skip if not a valid string

    # Find all citation tokens with their positions
    pattern = r"<MASK>.*?</MASK>"
    words = annotated_text.split()  # Tokenize by whitespace
    positions = [i for i, word in enumerate(words) if re.match(pattern, word)]
    '''
    for position in positions:
        print(position)
        print(words[position])
        '''

    # Total citation count
    total_citations = len(positions)

    # Count citation pairs within max_distance words
    nearby_citations = 0
    for i in range(len(positions) - 1):
        if (positions[i + 1] - positions[i]) <= max_distance:  # Check word distance
            nearby_citations += 1

    return total_citations, nearby_citations


In [4]:
test_row = df.iloc[0]
test_sec = test_row["sections_final"][0]["annotated_text"]
print(test_sec)

"[T]he rule of lenity[] teach[es] that ambiguities about the breadth of a cri minal statute should be resolved in the defendant's favor." <MASK>27c6ed5d-6a68-42a7-a3a1-a659a6852d7f</MASK> ( 2019). That rule has famously been described as "'perhaps not much less old than' the task of statut ory 'construction itself.'" Id. (quoting <MASK>920f9f16-d699-48ce-b443-30066ac7bb39</MASK>. At its core the rule of le nity is an understanding that "a fair warning should be given to the world in language that the common world will understand, of what the law intends to do if a certain line is passed," and that, "[t]o make the warning fair, so far as possible the line sh ould be clear." <MASK>d173dde0-afa7-4f58-8f50-c9bf5d70dc67</MASK>. In addition, judicial adherence to the rule of lenity also "places the weight of inertia upon the party" – that is, the executive branch – "that can best induce Congress to speak more clearly and keeps courts 7 from making criminal law in Congress's stead." <MASK>634

In [5]:
test_count = count_citations(test_sec)
print(test_count)

(9, 0)


In [6]:
total_citations = 0
total_nearby_citations = 0

indices = []
for i, row in df.iterrows():
    sections = row["sections_final"]

    if not isinstance(sections, list):
        continue  # Skip if sections_final is not a list

    for j, section in enumerate(sections):
        if isinstance(section, dict) and "annotated_text" in section:
            citations, nearby_citations = count_citations(section["annotated_text"])
            total_citations += citations
            total_nearby_citations += nearby_citations
            indices.append((i, j))

# Print the total counts
print(f"Total number of citations: {total_citations}")
print(f"Total number of nearby citations (within 5 words): {total_nearby_citations}")

Total number of citations: 86447
Total number of nearby citations (within 5 words): 11759


In [7]:
print(len(indices))

25733


In [8]:
index = 1
# The index of the row and section text are stored as a tuple in indices (i, j), because each row has multiple sections.
i = indices[index][0]
j = indices[index][1]
sections = df.iloc[i]["sections_final"]
print(sections[j]["annotated_text"])

1. The costs of erroneously construing mandatory minimum sentencing provisions too broadly are especially high, and the costs of construing them too narrowly are especially low. The greatest cost of reading a mandatory minimu m too broadly is that individual defendants lose their liberty. Mandatory minimum provisions are, by design, severe. They often tie an additional prison sentence of years or decades (here, a decade and a half) to a single factual determination. Such provisions therefore speak to the core concern that has motivated courts to apply the rule of lenity for centuries: the "instinctive 9 distaste[] against men lang uishing in prison unless the lawmaker has clearly sa id they should." Henry J. Friendly, "Mr. Justice Frankfurter and the Reading of Statutes," in Benchmarks 196, 209 (1967), quoted in <MASK>3e47527b-6426-41c6-bd2f-1555ae9a3dde</MASK>. Because of that concern, this Court "will not interpret a federal criminal statute so as to increase the penalty that it plac

In [9]:
total_citations = 0
total_nearby_citations = 0

# Iterate through each row and count citations
for _, row in df.iterrows():
    sections = row["sections_final"]

    if not isinstance(sections, list):
        continue  # Skip if sections_final is not a list

    for section in sections:
        if isinstance(section, dict) and "annotated_text" in section:
            citations, nearby_citations = count_citations(section["annotated_text"], max_distance=3)
            total_citations += citations
            total_nearby_citations += nearby_citations

# Print the total counts
print(f"Total number of citations: {total_citations}")
print(f"Total number of nearby citations (within 3 words): {total_nearby_citations}")

Total number of citations: 86447
Total number of nearby citations (within 3 words): 8759


In [10]:
total_citations = 0
total_nearby_citations = 0

# Iterate through each row and count citations
for _, row in df.iterrows():
    sections = row["sections_final"]

    if not isinstance(sections, list):
        continue  # Skip if sections_final is not a list

    for section in sections:
        if isinstance(section, dict) and "annotated_text" in section:
            citations, nearby_citations = count_citations(section["annotated_text"], max_distance=2)
            total_citations += citations
            total_nearby_citations += nearby_citations

# Print the total counts
print(f"Total number of citations: {total_citations}")
print(f"Total number of nearby citations (within 2 words): {total_nearby_citations}")

Total number of citations: 86447
Total number of nearby citations (within 2 words): 6101
